In [121]:
#!pip install -r requirements.txt
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [122]:
f = "data/tcga_brca_all_clean.csv.zip"

#TODO slide_id -> file path 

## Preprocessing genomic data 

In [162]:
n_bins = 4
df = pd.read_csv(f,compression='zip')

df_uncensored = (df[df["censorship"]==0]).drop_duplicates(["case_id"])
_,bins = pd.qcut(df_uncensored['survival_months'],q = n_bins,retbins=True)  # distribute censored survival months into quartiles

# adapt bins 
bins[0] = 0 
bins[-1] = np.inf
#
labels=[i for i in range(n_bins)]
labels

df.insert(6,"survival_months_discretized",  pd.cut(df["survival_months"],
                                                               bins=bins, 
                                                               labels=labels)) # insert binned survival momnth 
k = 7
df.insert(3,"kfold",df.index%k) # insert kfold 
df.head(5)

,case_id,slide_id,site,is_female,oncotree_code,age,survival_months_discretized,survival_months,censorship,train,...,ZWINT_rnaseq,ZXDA_rnaseq,ZXDB_rnaseq,ZXDC_rnaseq,ZYG11A_rnaseq,ZYG11B_rnaseq,ZYX_rnaseq,ZZEF1_rnaseq,ZZZ3_rnaseq,TPTEP1_rnaseq
0,TCGA-3C-AALI,TCGA-3C-AALI-01Z-00-DX1.F6E9A5DF-D8FB-45CF-B4B...,3C,1.0,IDC,50.0,3,131.57,1.0,1.0,...,0.3196,-0.1816,-0.4114,1.0797,1.0743,-1.0477,0.9689,-0.3256,-1.1244,2.4038
1,TCGA-3C-AALI,TCGA-3C-AALI-01Z-00-DX2.CF4496E0-AB52-4F3E-BDF...,3C,1.0,IDC,50.0,3,131.57,1.0,1.0,...,0.3196,-0.1816,-0.4114,1.0797,1.0743,-1.0477,0.9689,-0.3256,-1.1244,2.4038
2,TCGA-3C-AALJ,TCGA-3C-AALJ-01Z-00-DX1.777C0957-255A-42F0-9EE...,3C,1.0,IDC,62.0,2,48.42,1.0,1.0,...,3.2911,-1.1222,-0.1302,-1.4793,2.5267,-1.3026,0.9433,-1.1210,-0.9310,-0.6472
3,TCGA-3C-AALJ,TCGA-3C-AALJ-01Z-00-DX2.62DFE56B-B84C-40F9-962...,3C,1.0,IDC,62.0,2,48.42,1.0,1.0,...,3.2911,-1.1222,-0.1302,-1.4793,2.5267,-1.3026,0.9433,-1.1210,-0.9310,-0.6472
4,TCGA-3C-AALK,TCGA-3C-AALK-01Z-00-DX1.4E6EB156-BB19-410F-878...,3C,1.0,IDC,52.0,2,47.57,1.0,1.0,...,-0.2431,-0.3657,-0.4472,-1.0557,0.8232,-1.0342,1.0729,-1.4634,-0.5710,-0.5402


scale genomic data first and store in DF again
then add labels to stratify 

In [165]:
# todo add new column k-th-fold that contains an int in arange(k) where k is a cluster 
groundtruth = df["survival_months_discretized"]
censorship = df["censorship"]
genomics = df[df.keys()[11:]]
genomics


,NOTCH2_cnv,ACP6_cnv,ANKRD20A12P_cnv,ANKRD34A_cnv,ANKRD35_cnv,BCL9_cnv,BOLA1_cnv,CD160_cnv,CHD1L_cnv,FAM72B_cnv,...,ZWINT_rnaseq,ZXDA_rnaseq,ZXDB_rnaseq,ZXDC_rnaseq,ZYG11A_rnaseq,ZYG11B_rnaseq,ZYX_rnaseq,ZZEF1_rnaseq,ZZZ3_rnaseq,TPTEP1_rnaseq
403,1,1,1,1,1,1,1,1,1,1,...,-0.6752,-0.6708,-0.6352,0.3693,0.1446,3.2597,-0.1603,-0.8919,0.5903,-0.0801
598,0,0,0,0,0,0,0,0,0,0,...,-0.0525,-0.2132,-0.2256,0.1374,0.5972,-0.7959,-1.1744,-0.3666,-1.3888,-0.7535
817,0,0,0,0,0,0,0,0,0,0,...,0.3245,-0.3567,1.7451,-0.5448,1.7019,-0.6458,2.4399,0.7366,-0.3115,0.9021
244,0,0,0,0,0,0,0,0,0,0,...,-0.9956,-0.5694,-0.7817,0.0144,-0.9142,-1.2975,3.6863,0.0938,-0.0315,0.0677
320,1,1,1,1,1,1,1,1,1,1,...,1.2331,0.9744,-0.6189,-2.1771,1.8906,-0.3966,-0.0656,-1.6399,-0.5391,-0.6042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,1,1,1,1,1,1,1,1,1,1,...,-0.2865,-0.7057,-0.8934,0.2821,-0.1644,-1.1582,0.1480,0.0373,-0.6824,-0.5994
72,1,1,1,1,1,1,1,1,1,1,...,-0.2171,0.1621,-0.3724,1.5166,0.5926,1.0383,0.0326,-0.1507,0.2853,-0.3509
908,0,0,0,0,0,0,0,0,0,0,...,-0.5362,0.3350,0.2731,1.8174,0.8894,0.7809,-0.6663,-0.3758,0.2849,-0.6437
235,1,1,1,1,1,1,1,1,1,1,...,-0.9187,0.9464,0.0057,0.2020,0.0415,-0.1023,0.4811,0.3689,0.0824,-0.4666


In [169]:


scaler = StandardScaler()
scaled_genomics = scaler.fit_transform(genomics)
#scaler.inverse_transform(scaled_genomics)
df_scaled = df.copy()
df_scaled[df.keys()[11:]] = scaled_genomics



###  This leads to the full function:

In [123]:
def clean2trainable(df_path,k,n_bins=4,savename = None):
    df = pd.read_csv(df_path,compression='zip')

    # get time bins 
    df_uncensored = (df[df["censorship"]==0]).drop_duplicates(["case_id"])
    _,bins = pd.qcut(df_uncensored['survival_months'],q = n_bins,retbins=True)  # distribute censored survival months into quartiles

    # adapt time bins 
    bins[0] = 0 
    bins[-1] = np.inf
    # bin name = index 
    labels = [i for i in range(n_bins)]
    df.insert(6,"survival_months_discretized",  pd.cut(df["survival_months"],
                                                               bins=bins, 
                                                               labels=labels)) # insert binned survival month 
    df.insert(3,"kfold",df.index%k) # insert kfold 

    genomics = df[df.keys()[11:]]

    scaler = StandardScaler()
    scaled_genomics = scaler.fit_transform(genomics)
    df[df.keys()[11:]] = scaled_genomics

    if savename is not None:
        df.to_csv(savename,index=False)
    return df 

In [124]:
f = "data/tcga_brca_all_clean.csv.zip"
new_df = clean2trainable(f,5,4,savename = "./data/tcga_brca_trainable.csv")
#new_df = clean2trainable(f,7,4)

In [125]:
f_new = "data/tcga_brca_trainable.csv"
df = pd.read_csv(f_new)
df.head()

,case_id,slide_id,site,kfold,is_female,oncotree_code,age,survival_months_discretized,survival_months,censorship,...,ZWINT_rnaseq,ZXDA_rnaseq,ZXDB_rnaseq,ZXDC_rnaseq,ZYG11A_rnaseq,ZYG11B_rnaseq,ZYX_rnaseq,ZZEF1_rnaseq,ZZZ3_rnaseq,TPTEP1_rnaseq
0,TCGA-3C-AALI,TCGA-3C-AALI-01Z-00-DX1.F6E9A5DF-D8FB-45CF-B4B...,3C,0,1.0,IDC,50.0,3,131.57,1.0,...,0.153655,-0.207313,-0.427975,0.825128,0.825317,-0.963470,0.880672,0.164444,-1.064402,2.395788
1,TCGA-3C-AALI,TCGA-3C-AALI-01Z-00-DX2.CF4496E0-AB52-4F3E-BDF...,3C,1,1.0,IDC,50.0,3,131.57,1.0,...,0.153655,-0.207313,-0.427975,0.825128,0.825317,-0.963470,0.880672,0.164444,-1.064402,2.395788
2,TCGA-3C-AALJ,TCGA-3C-AALJ-01Z-00-DX1.777C0957-255A-42F0-9EE...,3C,2,1.0,IDC,62.0,2,48.42,1.0,...,2.665836,-1.099110,-0.184212,-1.279398,2.112450,-1.190473,0.856429,-0.579342,-0.874345,-0.485389
3,TCGA-3C-AALJ,TCGA-3C-AALJ-01Z-00-DX2.62DFE56B-B84C-40F9-962...,3C,3,1.0,IDC,62.0,2,48.42,1.0,...,2.665836,-1.099110,-0.184212,-1.279398,2.112450,-1.190473,0.856429,-0.579342,-0.874345,-0.485389
4,TCGA-3C-AALK,TCGA-3C-AALK-01Z-00-DX1.4E6EB156-BB19-410F-878...,3C,4,1.0,IDC,52.0,2,47.57,1.0,...,-0.322065,-0.381861,-0.459009,-0.931029,0.602789,-0.951448,0.979160,-0.899523,-0.520568,-0.384345


In [127]:
df[df.keys()[11:]].head()

,NOTCH2_cnv,ACP6_cnv,ANKRD20A12P_cnv,ANKRD34A_cnv,ANKRD35_cnv,BCL9_cnv,BOLA1_cnv,CD160_cnv,CHD1L_cnv,FAM72B_cnv,...,ZWINT_rnaseq,ZXDA_rnaseq,ZXDB_rnaseq,ZXDC_rnaseq,ZYG11A_rnaseq,ZYG11B_rnaseq,ZYX_rnaseq,ZZEF1_rnaseq,ZZZ3_rnaseq,TPTEP1_rnaseq
0,-2.117218,-2.241841,-2.241841,-2.241841,-2.241841,-2.241841,-2.241841,-2.241841,-2.241841,-2.241841,...,0.153655,-0.207313,-0.427975,0.825128,0.825317,-0.963470,0.880672,0.164444,-1.064402,2.395788
1,-2.117218,-2.241841,-2.241841,-2.241841,-2.241841,-2.241841,-2.241841,-2.241841,-2.241841,-2.241841,...,0.153655,-0.207313,-0.427975,0.825128,0.825317,-0.963470,0.880672,0.164444,-1.064402,2.395788
2,-0.863795,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,...,2.665836,-1.099110,-0.184212,-1.279398,2.112450,-1.190473,0.856429,-0.579342,-0.874345,-0.485389
3,-0.863795,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,...,2.665836,-1.099110,-0.184212,-1.279398,2.112450,-1.190473,0.856429,-0.579342,-0.874345,-0.485389
4,-0.863795,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,-0.929074,...,-0.322065,-0.381861,-0.459009,-0.931029,0.602789,-0.951448,0.979160,-0.899523,-0.520568,-0.384345


## k-Fold crossvalidation
1. Cluster dataset into k groups (shuffle first(seeded) then stratify by patient,split into equal chunks)
2. use one group for testing and the others for training
3. rotate such that each group is once used for testing
4. use distribution of testing results to estimate real value 



In [13]:
k = 7 
a = np.arange(k) # df k-th fold 
for i in range(k):  
    a = (a+1)%k
    print(f"train on {a[:-1]} test on {a[-1]}, store results in foldername: experiment1_fold{a[-1]}")



train on [1 2 3 4 5 6] test on 0, store results in foldername: experiment1_fold0
train on [2 3 4 5 6 0] test on 1, store results in foldername: experiment1_fold1
train on [3 4 5 6 0 1] test on 2, store results in foldername: experiment1_fold2
train on [4 5 6 0 1 2] test on 3, store results in foldername: experiment1_fold3
train on [5 6 0 1 2 3] test on 4, store results in foldername: experiment1_fold4
train on [6 0 1 2 3 4] test on 5, store results in foldername: experiment1_fold5
train on [0 1 2 3 4 5] test on 6, store results in foldername: experiment1_fold6


In [24]:
df = pd.DataFrame({"col1": np.arange(100),"col2":np.random.rand(100)})
df
df["col1"] = df["col1"].apply(lambda x: f"thisis{x}")
df

,col1,col2
0,thisis0,0.771627
1,thisis1,0.445847
2,thisis2,0.593957
3,thisis3,0.457410
4,thisis4,0.182458
...,...,...
95,thisis95,0.087985
96,thisis96,0.931732
97,thisis97,0.929314
98,thisis98,0.755483


In [28]:
f = "data/tcga_brca_all_clean.csv.zip"
f = f.replace("all_clean.csv.zip","trainable.csv")
f

'data/tcga_brca_trainable.csv'

In [29]:
!pip install torchmetrics


  Using cached torchmetrics-0.11.4-py3-none-any.whl (519 kB)

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [58]:
import torch 
idx = torch.randint(low=0,high=4,size=(100,))
values = torch.zeros(size=(100,4))
source = torch.ones(size=(100,))

In [59]:
values.scatter_(dim=1,index=idx,src=source)

RuntimeError: Index tensor must have the same number of dimensions as self tensor

In [104]:
groundtruth = torch.zeros(size=(10,4))
a,b = groundtruth.size()
labels = torch.randint(low=0,high=b,size=(1,a))
groundtruth[torch.arange(a),labels]=1
groundtruth

tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]])

In [105]:
from torchmetrics import Accuracy
acc = Accuracy("multiclass")

tensor([[1, 1, 1, 0, 0, 1, 2, 2, 3, 3]])

In [112]:
a = torch.randint(low=0,high=2,size=(100,))
a



tensor([1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 1])

In [120]:

from sksurv.metrics import concordance_index_censored
c = torch.randint(0,2,size=(100,))
l = torch.randint(0,4,size=(100,))
out = nn.Sigmoid()(torch.rand(size=(100,4)))
c_index = concordance_index_censored(c,l,out)


ValueError: Expected 1D array, got 2D array instead:
array=[[0.7033437  0.5068377  0.654999   0.5990905 ]
 [0.7150219  0.599375   0.7102213  0.669106  ]
 [0.64452285 0.5671275  0.71529394 0.6757698 ]
 [0.6388677  0.62970865 0.64201355 0.5307401 ]
 [0.60350907 0.72728115 0.53220797 0.518223  ]
 [0.68350834 0.6934904  0.5089255  0.61560374]
 [0.67109674 0.6994699  0.6647645  0.52406156]
 [0.55831575 0.64931524 0.5347419  0.65587   ]
 [0.52917504 0.6606084  0.55667764 0.65744233]
 [0.7292093  0.56855935 0.7007709  0.66647863]
 [0.6349134  0.6343174  0.57802343 0.67564833]
 [0.6474086  0.58789873 0.6473337  0.5406985 ]
 [0.6528097  0.55204225 0.5219494  0.588915  ]
 [0.5066584  0.575144   0.62465805 0.5473152 ]
 [0.6980143  0.5550397  0.69929653 0.67663085]
 [0.66518444 0.5020233  0.54762226 0.68529356]
 [0.61191684 0.57054913 0.51956755 0.5036999 ]
 [0.5990251  0.6647159  0.5022067  0.6415785 ]
 [0.5096261  0.5296232  0.64971465 0.5950778 ]
 [0.7218662  0.65232795 0.5983458  0.6257463 ]
 